In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#!/usr/bin/env python
# coding: utf-8
"""
@author: nakayama.s
"""

import os
import warnings
import gc
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from itertools import product

In [ ]:
def graph_insight(data):
    print(set(data.dtypes.tolist()))
    df_num = data.select_dtypes(include = ['float64', 'int64'])
    df_num.hist(figsize=(16, 16), bins=50, xlabelsize=8, ylabelsize=8);

def eda(data):
    # print(data)
    print("----------Top-5- Record----------")
    print(data.head(5))
    print("-----------Information-----------")
    print(data.info())
    print("-----------Data Types------------")
    print(data.dtypes)
    print("----------Missing value----------")
    print(data.isnull().sum())
    print("----------Null value-------------")
    print(data.isna().sum())
    print("----------Shape of Data----------")
    print(data.shape)
    print("----------describe---------------")
    print(data.describe())
    print("----------tail-------------------")
    print(data.tail())
    
def read_csv(path):
  # logger.debug('enter')
  df = pd.read_csv(path)
  # logger.debug('exit')
  return df

def load_train_data():
  # logger.debug('enter')
  df = read_csv(SALES_TRAIN_V2)
  # logger.debug('exit')
  return df

def load_test_data():
  # logger.debug('enter')
  df = read_csv(TEST_DATA)
  # logger.debug('exit')
  return df

def graph_insight(data):
    print(set(data.dtypes.tolist()))
    df_num = data.select_dtypes(include = ['float64', 'int64'])
    df_num.hist(figsize=(16, 16), bins=50, xlabelsize=8, ylabelsize=8);

def drop_duplicate(data, subset):
    print('Before drop shape:', data.shape)
    before = data.shape[0]
    data.drop_duplicates(subset,keep='first', inplace=True) #subset is list where you have to put all column for duplicate check
    data.reset_index(drop=True, inplace=True)
    print('After drop shape:', data.shape)
    after = data.shape[0]
    print('Total Duplicate:', before-after)

def unresanable_data(data):
    print("Min Value:",data.min())
    print("Max Value:",data.max())
    print("Average Value:",data.mean())
    print("Center Point of Data:",data.median())

SAMPLE_SUBMISSION    = '../input/sample_submission_V2.csv'
TRAIN_DATA           = '../input/train_V2.csv'
TEST_DATA            = '../input/test_V2.csv'

sample       = read_csv(SAMPLE_SUBMISSION)
train           = read_csv(TRAIN_DATA)
test            = read_csv(TEST_DATA)

In [ ]:
eda(train)

In [ ]:
eda(test)

In [ ]:
train = pd.get_dummies(train,columns=['matchType'])

In [ ]:
train =train.dropna()

In [ ]:
test = pd.get_dummies(test,columns=['matchType'])

In [ ]:
y_train =train['winPlacePerc']
x_train =train.drop(['Id','groupId','matchId','winPlacePerc'],axis=1)

In [ ]:
X_train = x_train.values
Y_train = y_train.values

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
from keras import models
from keras import layers
from keras import Sequential
from keras.layers import Dense, Dropout, Input

def build_model():
    # Because we will need to instantiate
    # the same model multiple times,
    # we use a function to construct it.
    model = Sequential()
    model.add(Dense(80,input_dim=X_train.shape[1],activation='relu'))
    model.add(Dense(160,activation='relu'))
    model.add(Dense(320,activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(160,activation='relu'))
    model.add(Dense(80,activation='relu'))
    model.add(Dense(40,activation='relu'))
    model.add(Dense(20,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
#     model.summary
#     history = model.fit(X_train, y_train, epochs=70,batch_size=100000)
    return model

In [ ]:
import numpy as np

k = 4
num_val_samples = len(X_train) // k
# num_epochs = 40
# all_scores = []
# for i in tqdm(range(k)):
#     print('processing fold #', i)
#     # Prepare the validation data: data from partition # k
#     val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
#     val_targets = Y_train[i * num_val_samples: (i + 1) * num_val_samples]

#     # Prepare the training data: data from all other partitions
#     partial_train_data = np.concatenate([X_train[:i * num_val_samples],X_train[(i + 1) * num_val_samples:]],axis=0)
#     partial_train_targets = np.concatenate([Y_train[:i * num_val_samples],Y_train[(i + 1) * num_val_samples:]],axis=0)

#     # Build the Keras model (already compiled)
#     model = build_model()
#     # Train the model (in silent mode, verbose=0)
#     model.fit(partial_train_data, partial_train_targets,
#               epochs=num_epochs, batch_size=100000, verbose=2)
#     # Evaluate the model on the validation data
#     val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=2)
#     all_scores.append(val_mae)

In [ ]:
# all_scores

In [ ]:
# np.mean(all_scores)

In [ ]:
from keras import backend as K

# Some memory clean-up
K.clear_session()

In [ ]:
from keras.callbacks import EarlyStopping 
 
# Early-stopping 
# early_stopping = EarlyStopping(patience=0, monitor='val_mean_absolute_error',verbose=2,) 
# callbacks=[early_stopping]

num_epochs = 200
all_mae_histories = []
for i in tqdm(range(k)):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = Y_train[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate([X_train[:i * num_val_samples],X_train[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = np.concatenate([Y_train[:i * num_val_samples],Y_train[(i + 1) * num_val_samples:]],axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_train_data, partial_train_targets,validation_data=(val_data, val_targets),epochs=num_epochs, batch_size=2048, verbose=2)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in tqdm(points):
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
# history.history['val_mean_absolute_error']
# history.history['val_loss']

In [ ]:
def plot_history(history):
    # print(history.history.keys())

    # 精度の履歴をプロット
    plt.plot(history.history['mean_absolute_error'])
    plt.plot(history.history['val_mean_absolute_error'])
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(['mean_absolute_error', 'val_mean_absolute_error'], loc='lower right')
    plt.show()

    # 損失の履歴をプロット
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['loss', 'val_loss'], loc='lower right')
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
x_test = test.drop(['Id','groupId','matchId'],axis=1)

In [ ]:
X_test = x_test.values

In [ ]:
X_test

In [ ]:
model = build_model()
# Train it on the entirety of the data.
model.fit(X_train, Y_train,epochs=120, batch_size=1024, verbose=1)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
sample['winPlacePerc'] = prediction

In [ ]:
sample

In [ ]:
sample.to_csv('sample_submission_v1.csv', index=False)